# Machine interpreter

## Instructions

Three registers, `a`, `b`, and `c`. 
Program counter, `pc`.
Unlimited memory in numbered addresses, completely separate from program storage.

Each register and memory location can hold 8-byte integers (Java `long`s).

Machine carries out the instruction at location `pc`. After it's executed, `pc` increments by 1.

`jmp` and `jpz` override this normal change in `pc`.

| Instruction | Description |
|:------------|:------------|
| `inc r`     | increment contents of register `r` |
| `dec r`     | decrement contents of register `r` |
| `set r i`   | set contents of register `r` to literal value `i` |
| `cpy r s`   | copy contents of register `r` into register `s` | 
| `sto r m`   | store contents of register `r` into memory location `m` |
| `ld r m`    | load contents of memory location `m` into register `r` |
| `jmp i`     | jump to instruction `i` places forward |
| `jpz r i`   | jump to instruction `i` places forward if<br>register `r` contains zero, otherwise continue to next instruction |

For `jmp` and `jpz`, `i` is relative to the current instruction. `i` can be negative to jump to earlier places in the program. `i`=1 is a no-op, `i`=0 causes an infinite loop.

In [1]:
def new_machine():
    return {'pc': 0, 
            'a': 0,
            'b': 0, 
            'c': 0,
            'instructions': []}

In [2]:
def show_machine(machine):
    return ', '.join('{}: {}'.format(sk, machine[int(sk) if sk.isnumeric() else sk]) 
                     for sk in sorted(str(k) for k in machine)
                     if sk != 'instructions')

In [3]:
def inc(reg, machine):
    machine[reg] += 1
    machine['pc'] += 1

In [4]:
def dec(reg, machine):
    machine[reg] -= 1
    machine['pc'] += 1

In [5]:
def jmp(addr, machine):
    machine['pc'] += addr

In [6]:
def jpz(reg, addr, machine):
    if machine[reg] == 0:
        machine['pc'] += addr
    else:
        machine['pc'] += 1

In [7]:
def set_literal(reg, literal, machine):
    machine[reg] = literal
    machine['pc'] += 1

In [8]:
def cpy(from_reg, to_reg, machine):
    machine[to_reg] = machine[from_reg]
    machine['pc'] += 1

In [9]:
def sto(reg, addr, machine):
    machine[addr] = machine[reg]
    machine['pc'] += 1

In [10]:
def ld(reg, addr, machine):
    if addr in machine:
        machine[reg] = machine[addr]
    else:
        machine[reg] = 0
    machine['pc'] += 1

In [11]:
instruction_table = {'inc': inc, 'dec': dec, 'jmp': jmp,
                    'jpz': jpz, 'set': set_literal, 'cpy': cpy,
                    'sto': sto, 'ld': ld}
numeric_args_table = {'jmp': [0], 'jpz': [1], 'set': [1], 'sto': [1], 'ld': [1]}

In [12]:
def parse(instruction):
    words = instruction.split()
    instr = words[0]
    args = words[1:]
    if instr in numeric_args_table:
        for p in numeric_args_table[instr]:
            args[p] = int(args[p])
    return instruction_table[instr], args

In [13]:
m = new_machine()
inc('a', m)
cargs = ['a', 'b']
cpy(*cargs, m)
inc('a', m)
m

{'a': 2, 'b': 1, 'c': 0, 'instructions': [], 'pc': 3}

In [14]:
def program_from_instructions(prog, machine):
    machine['instructions'] = [parse(instr) for instr in prog]

In [15]:
def program_from_listing(listing, machine):
    labelled_instructions = [i.strip() for i in listing.split('\n') 
                             if i.strip() 
                             if not i.strip().startswith('#')]
    instructions = replace_labels(labelled_instructions)
    program_from_instructions(instructions, machine)

In [16]:
def replace_labels(listing):
    locations = {}
    for n, i in enumerate(listing):
        if ':' in i:
            locations[i.split(':')[0]] = n

    unlabelled_listing = []
    for n, i in enumerate(listing):
        instr = i.split()
        if ':' in i:
            instr = i.split(':')[1].split()
        else:
            instr = i.split()
        terms = []
        for term in instr:
            if term in locations:
                terms += [str(locations[term] - n)]
            else:
                terms += [term]
        transformed_instr = ' '.join(terms)
        unlabelled_listing += [transformed_instr]
        
    return unlabelled_listing    

In [17]:
'fred: inc a'.split(':')[1].split()

['inc', 'a']

In [18]:
program = """
      set a 10
      # comment line
      
loop: dec a
      inc b
      sto b 1
      jpz a 2
      jmp loop
"""
labelled_instructions = [i.strip() for i in program.split('\n') if i.strip() if not i.strip().startswith('#')]
instructions = replace_labels(labelled_instructions)
print('\n'.join(instructions))

set a 10
dec a
inc b
sto b 1
jpz a 2
jmp -4


In [19]:
def run(machine, initial_state=None, trace=False):
    if initial_state:
        machine.update(initial_state)
    while machine['pc'] < len(machine['instructions']):
        if trace:
            print(show_machine(machine))
        cmd, args = machine['instructions'][machine['pc']]
        cmd(*args, machine)

In [20]:
def execute(listing, initial_state=None, trace=False):
    m = new_machine()
    program_from_listing(listing, m)
    run(m, initial_state=initial_state, trace=trace)
    return m

In [21]:
program = """
inc a
inc a
cpy a b
inc a
"""
execute(program)

{'a': 3,
 'b': 2,
 'c': 0,
 'instructions': [(<function __main__.inc>, ['a']),
  (<function __main__.inc>, ['a']),
  (<function __main__.cpy>, ['a', 'b']),
  (<function __main__.inc>, ['a'])],
 'pc': 4}

In [22]:
program = """
set a 10
dec a
inc b
sto b 1
jpz a 2
jmp -4
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
execute(program, initial_state={'c': 20})

{'pc': 6,
 'instructions': [(<function __main__.set_literal>, ['a', 10]),
  (<function __main__.dec>, ['a']),
  (<function __main__.inc>, ['b']),
  (<function __main__.sto>, ['b', 1]),
  (<function __main__.jpz>, ['a', 2]),
  (<function __main__.jmp>, [-4])],
 1: 10,
 'c': 20,
 'a': 0,
 'b': 10}

In [23]:
program = """
      set a 10
loop: dec a
      inc b
      sto b 1
      jpz a 2
      jmp loop
"""
execute(program, initial_state={'c': 20})

{'pc': 6,
 'instructions': [(<function __main__.set_literal>, ['a', 10]),
  (<function __main__.dec>, ['a']),
  (<function __main__.inc>, ['b']),
  (<function __main__.sto>, ['b', 1]),
  (<function __main__.jpz>, ['a', 2]),
  (<function __main__.jmp>, [-4])],
 1: 10,
 'c': 20,
 'a': 0,
 'b': 10}

In [24]:
program = """
cpy c a
set b 0
dec a
jpz b 3
dec b
jmp 2
inc b
jpz a 3
jmp -6
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
execute(program, initial_state={'c': 5})

{'a': 0,
 'b': 1,
 'c': 5,
 'instructions': [(<function __main__.cpy>, ['c', 'a']),
  (<function __main__.set_literal>, ['b', 0]),
  (<function __main__.dec>, ['a']),
  (<function __main__.jpz>, ['b', 3]),
  (<function __main__.dec>, ['b']),
  (<function __main__.jmp>, [2]),
  (<function __main__.inc>, ['b']),
  (<function __main__.jpz>, ['a', 3]),
  (<function __main__.jmp>, [-6])],
 'pc': 10}

In [25]:
# b holds parity of number in c: (c % 2)
program = """
      cpy c a
      set b 0
loop: dec a
      jpz b odd
      dec b
      jmp end
odd:  inc b
end:  jpz a 2
      jmp loop
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
execute(program, initial_state={'c': 5})

{'a': 0,
 'b': 1,
 'c': 5,
 'instructions': [(<function __main__.cpy>, ['c', 'a']),
  (<function __main__.set_literal>, ['b', 0]),
  (<function __main__.dec>, ['a']),
  (<function __main__.jpz>, ['b', 3]),
  (<function __main__.dec>, ['b']),
  (<function __main__.jmp>, [2]),
  (<function __main__.inc>, ['b']),
  (<function __main__.jpz>, ['a', 2]),
  (<function __main__.jmp>, [-6])],
 'pc': 9}

In [26]:
# c holds floor(a/2)
program = """
      set c 0
      set b 0
loop: dec a
      jpz b odd
      dec b
      inc c
      jmp end
odd:  inc b
end:  jpz a 2
      jmp loop
      sto c 1
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
show_machine(execute(program, initial_state={'a': 17}))

'1: 8, a: 0, b: 1, c: 8, pc: 11'

In [27]:
# c holds a * 3
program = """
      set c 0
      cpy a b
      # start of main loop
loop: jpz b end
      dec b
      inc c
      inc c
      inc c
      jmp loop
      
      # end of program 
      
end:  jmp 1
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
show_machine(execute(program, initial_state={'a': 4}))

'a: 4, b: 0, c: 12, pc: 9'

In [28]:
# c holds a * b
program = """
      set c 0
      sto a 1
loop: jpz b end      
      dec b
      ld a 1
smul: jpz a emul
      inc c
      dec a
      jmp smul
emul: jmp loop 
      
end:  sto a 1
"""
m = new_machine()
program_from_listing(program, m)
run(m)
show_machine(execute(program, initial_state={'a': 9, 'b': 3}))

'1: 0, a: 0, b: 0, c: 27, pc: 11'

In [29]:
# c holds a * b
program = """
      set a 2
      set b 5
      set c 0
      sto a 1
loop: jpz b end      
      dec b
      ld a 1
smul: jpz a emul
      inc c
      dec a
      jmp smul
emul: jmp loop 
      
end:  sto a 1
"""
m = new_machine()
program_from_listing(program, m)
run(m)
show_machine(execute(program, initial_state={'a': 9, 'b': 3}))

'1: 0, a: 0, b: 0, c: 10, pc: 13'

In [49]:
labelled_instructions = [i.strip() for i in program.split('\n') 
                             if i.strip() 
                             if not i.strip().startswith('#')]
instructions = replace_labels(labelled_instructions)
print('\n'.join(instructions))

set c 0
sto a 1
jpz b 8
dec b
ld a 1
jpz a 4
inc c
dec a
jmp -3
jmp -7
sto a 1


In [51]:
# Collatz. a initially holds value, but location 1 is used to store the current value as we're going along.
# Location 2 holds number of steps taken
# Location 3 holds max value reached
program = """
      sto a 1
      
      set b 0
      
      # if a is one, finish, 
main: dec a
      jpz a end
      inc a
      
      # find parity of a
      cpy a c
      set b 0
prty: dec c
      jpz b odd
      dec b
      jmp prte
odd:  inc b
prte: jpz c 2
      jmp prty
      
      # b == 0 means a even; b == 1 means a odd
      jpz b isev

      # c holds a * 3 + 1
      cpy a b
mul:  jpz b emul
      dec b
      inc c
      inc c
      inc c
      jmp mul
emul: inc c
      cpy c a
      jmp fin
      
      
isev: set c 0
      set b 0
hlvl: dec a
      jpz b oddh
      dec b
      inc c
      jmp endh
oddh: inc b
endh: jpz a 2
      jmp hlvl
      cpy c a

fin:  cpy a b
      ld  c 1
maxc: jpz c this
      jpz b othr
      dec b
      dec c
      jmp maxc
this: sto a 1
othr: jmp main
      
      # end of program 
      
end:  ld a 1
      set c 0
      sto c 1
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
show_machine(execute(program, initial_state={'a': 7}))

'1: 0, a: 52, b: 0, c: 0, pc: 48'

In [30]:
13*3+1

40

In [31]:
def max_collatz(start):
    mc = start
    i = start
    while i != 1:
        if i % 2 == 0:
            i = i // 2
        else:
            i = 3 * i + 1
        if i > mc:
            mc = i
    return mc

In [32]:
max_collatz(7)

52

In [33]:
max([(max_collatz(i), i) for i in range(1, 1000)])

(250504, 937)

In [34]:
ans = []
for i in range(1, 100):
    m = execute(program, initial_state={'a': i})
    c = max_collatz(i)
    if m[1] != c:
        ans += [(i, m[1], c)]
ans

[]

In [52]:
show_machine(execute(program, initial_state={'a': 937}))

'1: 0, a: 250504, b: 0, c: 0, pc: 48'

In [53]:
labelled_instructions = [i.strip() for i in program.split('\n') 
                             if i.strip() 
                             if not i.strip().startswith('#')]
instructions = replace_labels(labelled_instructions)
print('\n'.join(instructions))
open('07-program.txt', 'w').write('\n'.join(instructions))

sto a 1
set b 0
dec a
jpz a 42
inc a
cpy a c
set b 0
dec c
jpz b 3
dec b
jmp 2
inc b
jpz c 2
jmp -6
jpz b 11
cpy a b
jpz b 6
dec b
inc c
inc c
inc c
jmp -5
inc c
cpy c a
jmp 12
set c 0
set b 0
dec a
jpz b 4
dec b
inc c
jmp 2
inc b
jpz a 2
jmp -7
cpy c a
cpy a b
ld c 1
jpz c 5
jpz b 5
dec b
dec c
jmp -4
sto a 1
jmp -42
ld a 1
set c 0
sto c 1


342